Modules

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os

Load the data into a Pandas dataframe

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv')
df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,37.746,0.5,64.83,0.511,NaN,NaN,NaN,NaN


Create the engine and connect to it

In [3]:
username = os.environ.get('rds_covid_username')
password = os.environ.get('rds_covid_password')
host = os.environ.get('rds_covid_host')
dbname = 'covid_data'

conn = create_engine(f'mysql://{username}:{password}@{host}/{dbname}').connect()

Upload the dataframe on the database

In [4]:
df.to_sql(con = conn, name = 'covid', if_exists = 'replace', chunksize = 500, index = False)

Testing it with a query

In [5]:
query = '''SELECT EXTRACT(YEAR FROM CAST(date AS DATE)) year,
                  CONCAT(EXTRACT(MONTH FROM CAST(date AS DATE)), '-', EXTRACT(DAY FROM CAST(date AS DATE))) AS month_date
           FROM covid 
           WHERE location = 'Italy' '''

pd.read_sql_query(con = conn, sql = query)

,year,month_date
0,2020,1-31
1,2020,2-1
2,2020,2-2
3,2020,2-3
4,2020,2-4
...,...,...
694,2021,12-25
695,2021,12-26
696,2021,12-27
697,2021,12-28
